# 서로소 집합 (Disjoint Set)

## 집합의 표현

- 문제 출처: [백준 1717번](https://www.acmicpc.net/problem/1717)

`-` `disjoint-set`을 구현하여 해결할 수 있다

`-` 합집합은 두 집합을 합하면 그만이고 두 원소의 동일 집합 여부는 두 원소의 부모 노드가 동일한지로 판단할 수 있다

In [31]:
UNION = 0
CHECK = 1


def make_set(u):
    p[u] = u  # 각 노드가 자기자신을 가리키게 한다 (u -> u)
    rank[u] = 0


def find_set(u):  # u가 포함된 tree의 부모 노드를 찾아준다
    if p[u] != u:  # u가 자기자신을 가리키지 않으면 (=자식 노드)
        p[u] = find_set(p[u])  # flatten tree, original: (1 -> 3, 3 -> 5, 5 -> 7, 7 -> 7), new: (1 -> 7, 3 -> 7, 5 -> 7, 7 -> 7)
    return p[u]


def union_set(u, v):
    uu = find_set(u)
    vv = find_set(v)
    if uu == vv:  # uu와 vv가 같다면 이미 같은 tree에 속하므로 union할 이유가 없다
        return
    rank_u = rank[uu]
    rank_v = rank[vv]
    if rank_u > rank_v:  # v -> u
        p[vv] = uu
    elif rank_u == rank_v:  # v -> u(u -> v도 가능) and rank에 +1
        p[vv] = uu
        rank[vv] += 1
    else:  # u -> v
        p[uu] = vv


def solution():
    global p, rank
    n, m = map(int, input().split())
    p = [i for i in range(n + 1)]
    rank = [0 for _ in range(n + 1)]
    for i in range(n + 1):  # 이미 p를 make_set 적용한 상태로 만들어서 안해도 상관없지만 의미를 분명하게 하려고 추가함
        make_set(i)
    for _ in range(m):
        operator, a, b = map(int, input().split())
        if operator == UNION:
            union_set(a, b)
        else:
            if find_set(a) == find_set(b):
                print("YES")
            else:
                print("NO")


solution()

# input
# 7 8
# 0 1 3
# 1 1 7
# 0 7 6
# 1 7 1
# 0 3 7
# 0 4 2
# 0 1 1
# 1 1 1

 7 8
 0 1 3
 1 1 7


NO


 0 7 6
 1 7 1


NO


 0 3 7
 0 4 2
 0 1 1
 1 1 1


YES


## 거짓말

- 문제 출처: [백준 1043번](https://www.acmicpc.net/problem/1043)

`-` 이 문제는 `union-find` 알고리즘을 통해 해결할 수 있다

`-` 같은 파티에 있는 사람들은 한 배를 탄 것이다

`-` 그 사람들 전체가 진실을 몰라야 해당 파티에서 거짓말을 할 수 있다

`-` 누구 하나라도 진실을 알면 해당 파티에 있는 모든 사람들에게 더 이상 거짓말을 할 수 없다

`-` 그 사람들은 진실을 아는 사람이 되었기 때문에 또 다른 파티에 원래 진실을 아는 사람이 없어도 거짓말을 할 수 없다

`-` 처음에 사람들을 개별 집합으로 초기화한다

`-` 그리고 같은 파티에 있는 사람들을 합친다

`-` 두 명씩 합치면 되며 합집합 연산을 할 때마다 같은 그룹 사람이 한 명 늘어나므로 파티에 $N$명이 있다면 합집합 연산을 $N-1$번 하면 된다

`-` 모든 파티에 대해 합집합 연산을 끝낸 후 원래 진실을 아는 사람을 고려하자

`-` 원래 진실을 아는 사람 각각에 대해 그가 속한 트리의 루트 노드를 set에 추가한다

`-` 파티 하나에 대해 각 참여자들이 속한 트리의 루트 노드가 원래 진실을 아는 사람들 집합에 포함되어 있는지 확인한다

`-` 단 한명이라도 포함되어 있다면 그 파티에서 거짓말을 할 수 없다

`-` 이를 모든 파티에 대해 반복하면 거짓말을 할 수 있는 파티 수의 최댓값을 계산할 수 있다

In [13]:
def make_set(u):
    parent[u] = u
    rank[u] = 0


def find(u):
    if parent[u] != u:
        parent[u] = find(parent[u])
    return parent[u]


def union(u, v):
    root_u = find(u)
    root_v = find(v)
    if root_u == root_v:  # 이미 같은 집합에 속해 있다
        return
    rank_u = rank[root_u]
    rank_v = rank[root_v]
    if rank_u > rank_v:
        parent[root_v] = root_u
    elif rank_u < rank_v:
        parent[root_u] = root_k
    else:
        parent[root_v] = root_u
        rank[root_v] += 1


def solution():
    global parent, rank
    N, M = map(int, input().split())
    parent = [0 for _ in range(N + 1)]  # p[u]는 u가 가리키고 있는 부모 노드
    rank = [0 for _ in range(N + 1)]  # rank[u]는 u가 속한 트리 집합 높이의 상한
    for i in range(1, N + 1):
        make_set(i)
    true_people = list(map(int, input().split()))
    true_people.pop(0)  # 진실을 아는 사람의 수 (필요 없음)
    participants_list = []
    for _ in range(M):
        participants = list(map(int, input().split()))
        n = participants.pop(0)  # 파티에 참여한 사람의 수
        participants_list.append(participants)
        for i in range(n - 1):
            union(participants[i], participants[i + 1])
    true_set = set()
    for t in true_people:
        true_set.add(find(t))
    answer = 0
    for participants in participants_list:
        can_lie = True
        for p in participants:
            if find(p) in true_set:
                can_lie = False
        if can_lie:
            answer += 1
    print(answer)


solution()

# input
# 10 9
# 4 1 2 3 4
# 2 1 5
# 2 2 6
# 1 7
# 1 8
# 2 7 8
# 1 9
# 1 10
# 2 3 10
# 1 4

 10 9
 4 1 2 3 4
 2 1 5
 2 2 6
 1 7
 1 8
 2 7 8
 1 9
 1 10
 2 3 10
 1 4


4


## 사이클 게임

- 문제 출처: [백준 20040번](https://www.acmicpc.net/problem/20040)

`-` 두 점을 연결한다는 것은 두 점 각각과 연결된 원소로 이루어진 집합을 결합한다는 의미이다

`-` 이는 `union-find` 알고리즘을 사용해 해결할 수 있다

`-` 계속해서 주어지는 두 점을 `union` 해나간다

`-` 만약 이미 두 점이 하나의 집합에 포함되어 있다면 사이클이 완성된 것이다 (이는 루트 노드의 일치 여부로 판단 가능하다)

In [14]:
def make_set(u):
    p[u] = u
    rank[u] = 0


def find(u):
    if u != p[u]:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return "no"
    if rank[u_root] < rank[v_root]:
        p[u_root] = v_root
    elif rank[v_root] < rank[u_root]:
        p[v_root] = u_root
    else:
        p[u_root] = v_root
        rank[v_root] += 1
    return "yes"


def solution():
    global p, rank
    n, m = map(int, input().split())
    INF = 2e6
    p = [i for i in range(n)]
    rank = [0 for _ in range(n)]
    for i in range(n):
        make_set(i)
    answer = INF
    for i in range(1, m + 1):
        a, b = map(int, input().split())
        success = union(a, b)
        if success == "yes":
            continue
        answer = min(i, answer)
    if answer == INF:
        answer = 0
    print(answer)


solution()

# input
# 6 5
# 0 1
# 1 2
# 1 3
# 0 3
# 4 5

 6 5
 0 1
 1 2
 1 3
 0 3
 4 5


4


## 여행 가자

- 문제 출처: [백준 1976번](https://www.acmicpc.net/problem/1976)

`-` `disjoint-set`을 사용해 해결할 수 있다

`-` 입력으로 주어진 두 도시를 `union`하고 마지막에 여행을 계획한 도시가 같은 집합에 속하면 여행 가능하다

In [9]:
def make_set(u):
    p[u] = u
    rank[u] = 0


def find(u):
    if p[u] != u:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if rank[v_root] < rank[u_root]:
        p[v_root] = u_root
    elif rank[u_root] < rank[v_root]:
        p[u_root] = v_root
    else:
        p[u_root] = v_root
        rank[u_root] += 1


def solution():
    global p, rank
    N = int(input())
    M = int(input())
    p = [i for i in range(N + 1)]
    rank = [0 for _ in range(N + 1)]
    for u in range(1, N + 1):
        make_set(u)
    for i in range(1, N + 1):
        connection_info = map(int, input().split())
        for j, is_connect in enumerate(connection_info, start=1):
            if is_connect:
                union(i, j)
    cities = list(map(int, input().split()))  # 여행 계획 도시
    answer = "YES"
    root = find(cities[0])
    for city in cities:
        if find(city) != root:
            answer = "NO"
            break
    print(answer)


solution()

# input
# 3
# 3
# 0 1 0
# 1 0 1
# 0 1 0
# 1 2 3

 3
 3
 0 1 0
 1 0 1
 0 1 0
 1 2 3


YES


## 친구 네트워크

- 문제 출처: [백준 4195번](https://www.acmicpc.net/problem/4195)

`-` 서로소 집합을 사용하되 두 집합을 합칠 때 집합의 크기도 합치면 된다

`-` 두 집합을 합칠 때 랭크에 따라 한 집합의 루트 노드를 다른 집합의 루트 노드에 연결한다

`-` 랭크뿐만 아니라 집합의 크기도 기록하는 배열을 만들고 `union`할 때 처리하자

`-` 숫자가 아니라 문자열이 노드를 나타내므로 배열 대신 딕셔너리를 사용하자

In [3]:
def make_set(u):
    p[u] = u
    rank[u] = 0
    size[u] = 1


def find(u):
    if p[u] != u:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if rank[v_root] < rank[u_root]:
        p[v_root] = u_root
        size[u_root] += size[v_root]
    elif rank[u_root] < rank[v_root]:
        p[u_root] = v_root
        size[v_root] += size[u_root]
    else:
        p[u_root] = v_root
        rank[u_root] += 1
        size[v_root] += size[u_root]
    

def solve_testcase():
    global p, rank, size
    F = int(input())
    p = {}
    rank = {}
    size = {}
    for _ in range(F):
        a, b = input().split()
        if a not in p:
            make_set(a)
        if b not in p:
            make_set(b)
        union(a, b)
        print(size[find(a)])  # a와 b는 같은 집합이므로 find(a)와 find(b)는 동일함


def solution():
    T = int(input())
    for _ in range(T):
        solve_testcase()


solution()

# input
# 2
# 3
# Fred Barney
# Barney Betty
# Betty Wilma
# 3
# Fred Barney
# Betty Wilma
# Barney Betty

 2
 3
 Fred Barney


2


 Barney Betty


3


 Betty Wilma


...

2


 Betty Wilma


2


 Barney Betty


4


## 벽 부수고 이동하기 4

- 문제 출처: [백준 16946번](https://www.acmicpc.net/problem/16946)

`-` 간단한 방법은 각 벽에 대해 dfs를 수행하여 이동할 수 있는 칸의 개수를 세는 것이다

`-` $N\times M$의 행렬로 표현되는 맵의 노드는 $N+M$개이고 간선은 $2NM-N-M$개이다

`-` 맵에 벽이 많으면 dfs를 많이 수행하는 대신 이동 반경이 적어진다

`-` 맵의 테두리에 벽이 있다고 하면 벽은 $2N+2M-4$개 존재한다

`-` 각 벽에 대해 dfs를 수행하는 것은 $O\left(N^2M+NM^2\right)$의 시간 복잡도를 가지고 $N,M\le 1000$이므로 시간 초과이다

`-` 맵에 벽이 $2$개 존재한다고 해보자

`-` 하나의 벽에 대해 dfs를 수행하고 나머지 벽에 대해 dfs를 수행한다고 해보자

`-` 이전의 dfs 결과를 바탕으로 어떤 좌표끼리 연결되어 있는지 안다

`-` 나머지 벽을 제외한 모든 공간이 연결되어 있으므로 두 번째 dfs를 수행할 때 모든 공간을 탐색할 필요가 없다

`-` 즉, 공간이 연결되었다는 것은 해당 공간에 속한 임의의 좌표를 방문할 수 있으면 나머지 공간을 모두 방문할 수 있다는 뜻이다

`-` 맵의 각 빈칸에 대해 dfs를 수행하며 방문하는 곳을 하나의 집합으로 관리하자

`-` 이를 union-find 알고리즘을 통해 수행할 것이다

`-` 일단 $2$차원 좌표를 $1$차원 번호로 변환하자

`-` 좌표가 $(x,y)$라면 새로운 번호는 $My + x$이다 ($x,y$는 $0$부터 시작)

`-` 그럼 번호는 $0$부터 $NM-1$까지이다

`-` 벽이 아닌 각 번호에 대해 방문하지 않았다면 dfs를 수행하자

`-` 임의의 공간에 속한 벽이 아닌 좌표 개수를 알기 위해 size 배열을 사용하자

`-` dfs를 수행하면서 만나는 번호에 대해 union을 수행하고 방문 체크를 하자

`-` 만난 번호가 다른 집합이라면 union 과정에서 size도 병합해줘야 한다

`-` 이제 각 벽에 대해 해당 벽을 부수고 이동할 수 있는 곳으로 바꾸고 그 위치에서 이동할 수 있는 칸의 개수를 세어보자

`-` 이는 벽을 기준으로 상하좌우에 위치한 벽이 아닌 공간의 집합의 size 개수 합에 $1$을 더한 것이다 (중복 집합은 제외)

`-` 위와 같이 하면 탐색한 공간은 다시 탐색하지 않으므로 시간 복잡도는 $O(NM)$이 된다

`-` 계속 틀려서 질문 게시판의 반례를 찾아봤다

`-` 메모리 아낄려고 벽 부순 정보의 그래프를 원래 그래프에 덮어 씌우는 방식을 사용했다

`-` 대부분의 경우엔 문제가 안되는데 $10$으로 나눈 나머지가 $0$이면 문제가 된다

`-` 원래는 벽이지만 $0$으로 덮어씌워져서 움직일 수 있는 공간이 되고 이는 인접한 벽에 영향을 끼친다

`-` 모든 벽에 대해 움직일 수 있는 개수를 센 뒤 순회를 다시 하면서 $10$으로 나눴다

In [1]:
import sys

sys.setrecursionlimit(10**6 + 2)


def make_set(u):
    p[u] = u
    rank[u] = 0
    size[u] = 1  # 자기 자신


def find(u):
    if u != p[u]:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if rank[u_root] < rank[v_root]:
        p[u_root] = v_root
        size[v_root] += size[u_root]
    elif rank[u_root] > rank[v_root]:
        p[v_root] = u_root
        size[u_root] += size[v_root]
    else:
        p[v_root] = u_root
        size[u_root] += size[v_root]
        rank[u_root] += 1


def to_1d(x, y):
    return M * y + x


def dfs(x, y, graph, visited):
    num = to_1d(x, y)
    visited.add(num)
    for dx, dy in dxy:
        x_next = x + dx
        y_next = y + dy
        num_next = to_1d(x_next, y_next)
        if num_next in visited:
            continue
        is_in_range = 0 <= x_next < M and 0 <= y_next < N
        need_to_move = is_in_range and graph[y_next][x_next] == BLANK
        if not need_to_move:
            continue
        union(num, num_next)
        dfs(x_next, y_next, graph, visited)


def init_set(graph):
    for x in range(M):
        for y in range(N):
            if graph[y][x] == WALL:
                continue
            num = to_1d(x, y)
            make_set(num)


def classify_area(graph):
    visited = set()
    for x in range(M):
        for y in range(N):
            if graph[y][x] == WALL:
                continue
            num = to_1d(x, y)
            if num in visited:
                continue
            dfs(x, y, graph, visited)
    return visited


def aggregate_move_count(graph):
    for x in range(M):
        for y in range(N):
            if graph[y][x] == BLANK:
                continue
            move_count = 1
            visited = set()
            for dx, dy in dxy:
                x_next = x + dx
                y_next = y + dy
                is_in_range = 0 <= x_next < M and 0 <= y_next < N
                if not is_in_range:
                    continue
                if graph[y_next][x_next] != BLANK:
                    continue
                num = to_1d(x_next, y_next)
                root = find(num)
                if root in visited:
                    continue
                visited.add(root)
                move_count += size[root]
            graph[y][x] = move_count
    for x in range(M):
        for y in range(N):
            graph[y][x] %= 10
    return graph


def solution():
    global N, M, BLANK, WALL, dxy, p, rank, size
    N, M = map(int, input().split())
    graph = [list(map(int, list(input()))) for _ in range(N)]
    dxy = [(0, -1), (0, 1), (-1, 0), (1, 0)]
    BLANK = 0
    WALL = 1
    p = [u for u in range(N * M)]
    rank = [0 for _ in range(N * M)]
    size = [1 for _ in range(N * M)]
    init_set(graph)
    visited = classify_area(graph)
    graph = aggregate_move_count(graph)
    for row in graph:
        print("".join(map(str, row)))


solution()

# input
# 4 5
# 11001
# 00111
# 01010
# 10101

 4 5
 11001
 00111
 01010
 10101


46003
00732
06040
50403


## 공항

- 문제 출처: [백준 10775번](https://www.acmicpc.net/problem/10775)

`-` $i$번째 비행기를 $1$번부터 $g_i$번까지의 게이트 중 하나에 도킹할 수 있다

`-` 비행기 도킹에 실패하면 공항이 폐쇄되며 이후 비행기 도킹이 불가능해진다

`-` 따라서 현재의 비행기를 무조건 도킹시켜야 비행기를 최대한 많이 도킹할 수 있다

`-` $1$부터 $g_i$까지 가능하므로 무조건 큰 번호에 도킹해야 한다

`-` $g_i$번 게이트는 현재 비행기만 도킹 가능할지 몰라도 $1$번은 모든 비행기가 도킹 가능하다

`-` 어차피 게이트는 한 번 도킹되면 사용 불가능 하므로 지금 $g_i$ 게이트에 도킹 안하면 다음에 $g_i$ 게이트에 도킹시켜야 하는데 그럼 손해다

`-` 어차피 $g_i$ 게이트에 도킹하는데 지금 안할 이유가 없다

`-` 더 정확히 말하자면 $1$부터 $g_i$까지의 도킹 가능한 게이트 중 가장 번호가 큰 게이트에 도킹해야 한다

`-` 나이브한 방법으론 번호가 큰 게이트부터 순회해서 도킹 가능하면 도킹하고 방문 표시 하는거다

`-` 이는 게이트 수를 $G$, 비행기의 수를 $P$라 할 때 $O(GP)$의 시간 복잡도를 가지고 $G, P \le 10^5$이므로 시간 초과이다

`-` $1 \le x \le g_i$중 도킹 가능한 번호만 고려했을 때 최댓값을 골라야 한다

`-` 연속된 구간이 도킹됐다면 해당 구간 하나로 관리하고 이를 통째로 스킵하자

`-` 각 게이트를 분리 집합으로 관리하고 $g$번 게이트에 도킹하면 인접한 게이트인 $g-1, g+1$과 union하자

`-` 그리고 도킹된 각 집합은 게이트 번호의 최솟값을 지니고 있어 해당 집합에 진입하면 바로 최솟값까지 스킵하자

`-` $\min - 1$번 게이트가 존재하면 해당 게이트에 도킹하면 된다

`-` 반례보고 질문검색 글 몇 개 정독하고 왔다

`-` 예제랑 반례는 다 맞는데 계속 `2%`에서 틀린다

`-` 정신이 나갈 것 같지만 곰곰이 생각해봤다

`-` 그룹에 속하는 게이트에 도킹한 후 해당 그룹에서 $\min - 1$번 게이트를 선택해야 된다

`-` $\min - 1$과 그룹을 연결시켜야 한다

`-` 여기까진 당연한데 그 이후에 또 처리를 해야 한다

`-` 바로 $\min - 2$와 $\min - 1$이 연결될 가능성이 있다는 것이다

`-` 이 둘을 연결하는걸 깜빡해서 계속 틀렸다;;

`-` 또 틀렸는데 반례 찾았다

`-` 조건문에서 `docking[g_root]`을 사용했는데 이게 $\min - 1$과 그룹 연결할 때 동적으로 변한다

`-` 그래서 $\min - 2$와 $\min - 1$을 연결할 때 `docking[g_root] - 2`, `docking[g_root] - 1`이 아닌 `docking[g_root] - 1`, `docking[g_root]`을 연결해야 된다

`-` 이게 싫으면 `docking[g_root]`을 변수로 저장하고 해당 변수를 사용해야 된다

`-` 반례 찾고 디버깅해서 맞혔다

In [3]:
def make_set(u):
    p[u] = u
    rank[u] = 0


def find(u):
    if p[u] != u:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if docking[u_root] == NOT_YET or docking[v_root] == NOT_YET:
        return
    if docking[u_root] < docking[v_root]:
        min_ = docking[u_root]
    else:
        min_ = docking[v_root]
    if rank[v_root] < rank[u_root]:
        p[v_root] = u_root
        docking[u_root] = min_
    elif rank[u_root] < rank[v_root]:
        p[u_root] = v_root
        docking[v_root] = min_
    else:
        p[u_root] = v_root
        rank[u_root] += 1
        docking[v_root] = min_


def dock_airplane(docking):
    docking_count = 0
    answer = 0
    END = False
    for _ in range(P):
        g = int(input())
        if END:
            continue
        g_root = find(g)
        if docking[g_root] == NOT_YET:
            docking[g_root] = g_root
            if g_root < G:
                union(g_root, g_root + 1)
            if g_root > 1:
                union(g_root - 1, g_root)
            docking_count += 1
            answer = docking_count
            continue
        if docking[g_root] > 1:
            docking[docking[g_root] - 1] = docking[g_root] - 1
            union(docking[g_root] - 1, docking[g_root])
            docking_count += 1
            answer = docking_count
            if docking[g_root] > 1:
                union(docking[g_root] - 1, docking[g_root])
            continue
        if not END:
            answer = docking_count
            END = True
    return answer


def solution():
    global G, P, NOT_YET, p, rank, docking
    G = int(input())
    P = int(input())
    NOT_YET = -1
    p = [u for u in range(G + 1)]
    rank = [0 for _ in range(G + 1)]
    for u in range(1, G + 1):
        make_set(u)
    docking = [NOT_YET for _ in range(G + 1)]
    answer = dock_airplane(docking)
    print(answer)


solution()

# input
# 4
# 3
# 4
# 1
# 1

 4
 3
 4
 1
 1


2


## 닭싸움 팀 정하기

- 문제 출처: [백준 1765번](https://www.acmicpc.net/problem/1765)

`-` 친구의 친구도 친구고 원수의 원수도 친구다

`-` 인간관계를 바탕으로 그룹 짓고 최종적으로 그룹의 개수를 출력하면 된다

`-` 친구면 같은 팀이고 같은 팀이면 친구여야 하니 친구끼리만 모두 모아서 팀을 만들면 된다

`-` 그룹 관리는 분리 집합을 사용하여 수행하자

`-` 친구의 친구인 케이스는 그 둘을 union하면 된다

`-` 원수의 원수인 케이스는 약간의 프로세스가 필요하다

`-` 일단 둘은 서로 원수이므로 같은 팀일 수가 없다 (입력에 모순은 없다)

`-` 각 사람의 원수를 배열로 관리하자

`-` 원수 관계가 주어지면 $x, y$에 대해 $y$의 원수 배열을 순회하며 $x$와 union한다

`-` 그리고 $x$의 원수 배열을 순회하며 $y$와 union한다

`-` 그리고 $x$의 원수 배열에 $y$를 추가하고 $y$의 원수 배열에 $x$를 추가하면 된다

`-` 최악의 경우 원수 배열 순회는 $O(n)$이고 이를 $m$번 반복하므로 $O(nm)$이다

`-` 그런데 $n \le 1000, m \le 5000$이므로 제한 시간 안에 해결할 수 있다

In [5]:
def find(u):
    if u != parents[u]:
        parents[u] = find(parents[u])
    return parents[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if ranks[u_root] < ranks[v_root]:
        parents[u_root] = v_root
    elif ranks[u_root] > ranks[v_root]:
        parents[v_root] = u_root
    else:
        parents[v_root] = u_root
        ranks[u_root] += 1


def make_teams(n, m):
    man2enemies = [[] for _ in range(n + 1)]
    for _ in range(m):
        operator, p, q = input().split()
        p, q = int(p), int(q)
        if operator == FRIEND:
            union(p, q)
            continue
        for e in man2enemies[p]:
            union(q, e)
        for e in man2enemies[q]:
            union(p, e)
        man2enemies[p].append(q)
        man2enemies[q].append(p)
    num_teams = 0
    visited = set()
    for u in range(1, n + 1):
        root = find(u)
        if root in visited:
            continue
        num_teams += 1
        visited.add(root)
    return num_teams


def solution():
    global FRIEND, ENEMY, parents, ranks
    FRIEND = "F"
    ENEMY = "E"
    n = int(input())
    m = int(input())
    parents = [u for u in range(n + 1)]
    ranks = [0 for _ in range(n + 1)]
    num_teams = make_teams(n, m)
    print(num_teams)


solution()

# input
# 6
# 4
# E 1 4
# F 3 5
# F 4 6
# E 1 2

 6
 4
 E 1 4
 F 3 5
 F 4 6
 E 1 2


3


## 트리

- 문제 출처: [13306번](https://www.acmicpc.net/problem/13306)

`-` 단계별로 풀어보기 - 유니온 파인드 $2$에 있는 문제이다

`-` 유니온 파인드는 간선을 추가하는 연산만 지원하는데 여기선 어떻게 할까요?가 이 문제의 힌트이다

`-` 쿼리를 보면 특정 간선을 제거하는 것과 두 노드가 연결되었는지 물어보는 것이 있다

`-` 두 노드가 연결됐는지는 $O(N)$에 판단 가능한데 이것이 $Q$개 존재하므로 $O(NQ)$의 시간 복잡도를 가진다

`-` $N,Q \le 200000$이므로 나이브한 방법은 사용 불가능하다 

`-` 어떻게 푸는지 모르겠어서 태그를 봤는데 `오프라인 쿼리`를 사용한다고 한다

`-` 검색해보니 오프라인 쿼리는 쿼리가 주어지면 바로 처리를 하는게 아니라 이를 모으고 특정 작업을 거쳐 처리한다고 한다

`-` 근데 이 문제랑 무슨 상관이지?

`-` 두 노드 $u,v$가 연결되었는지는 어떤 간선을 끊냐에 따라 다르다

`-` 즉, 모아서 처리가 불가능하고 인터랙티브하게 그때 그때 처리해야 된다

`-` 근데 왜 태그엔 오프라인 쿼리가 있을까? ---> 그래서 문제를 잘 읽어봤다

`-` 간설을 끊는 쿼리 $N-1$개 주어지고 두 노드의 연결 여부 쿼리는 $Q$개 주어진다 (이걸 이제 알았네;)

`-` 트리의 간선은 $N-1$개이므로 쿼리를 진행하다보면 결국 모든 간선이 제거된다

`-` 쿼리를 전부 모은 다음 역순으로 쿼리를 수행하자

`-` find 연산을 통해 두 노드의 루트가 같으면 yes 아니면 no이다

`-` 간선을 끊는 쿼리가 보이면 두 노드를 연결하자

`-` 해당 쿼리 이후엔 두 노드가 끊어져 있지만 쿼리 이전에는 연결되어 있었다

`-` 이걸 처음 쿼리까지 반복한 뒤 연결 여부 쿼리의 대답을 역순으로 출력하면 정답이다

`-` 오늘의 교훈: 문제를 잘 읽자

`-` 쿼리를 역순으로 처리해서 간선을 끊는 작업을 합치는 작업으로 바꾼게 신박했다

`-` 오프라인 쿼리 재밌네

In [4]:
def find(u):
    if u != parents[u]:
        parents[u] = find(parents[u])
    return parents[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if ranks[u_root] < ranks[v_root]:
        parents[u_root] = v_root
    elif ranks[u_root] > ranks[v_root]:
        parents[v_root] = u_root
    else:
        parents[v_root] = u_root
        ranks[u_root] += 1


def offline_query(queries, parent_nodes):
    queries.reverse()
    answers = []
    for query in queries:
        if len(query) == 2:
            _, b = query
            union(b, parent_nodes[b])
        else:
            _, c, d = query
            if find(c) == find(d):
                answers.append("YES")
            else:
                answers.append("NO")
    return answers


def solution():
    global parents, ranks
    N, Q = map(int, input().split())
    parent_nodes = [None for _ in range(N + 1)]
    for i in range(1, N):
        a = int(input())
        parent_nodes[i + 1] = a
    parents = [u for u in range(N + 1)]
    ranks = [0 for _ in range(N + 1)]
    queries = [list(map(int, input().split())) for _ in range(N + Q - 1)]
    answers = offline_query(queries, parent_nodes)
    print("\n".join(answers[::-1]))


solution()

# input
# 3 3
# 1
# 1
# 1 2 3
# 0 3
# 1 2 3
# 1 1 2
# 0 2

 3 3
 1
 1
 1 2 3
 0 3
 1 2 3
 1 1 2
 0 2


YES
NO
YES


## 트리의 색깔과 쿼리

- 문제 출처: [백준 17469번](https://www.acmicpc.net/problem/17469)

`-` 오프라인 쿼리(13306번) 문제와 스몰 투 라지(28277번) 문제를 합친 문제이다

`-` 쿼리를 역순으로 처리하면 초기에는 모든 간선이 다 끊겨있다

`-` 간선 제거 쿼리가 나오면 두 노드를 union하면서 각 노드가 가지고 있는 색깔 집합을 하나로 합치자

`-` 합칠 때 스몰 투 라지 기법을 사용하면 모든 집합을 합쳐 하나의 집합으로 만드는데 $O(N \log N)$의 시간 복잡도로 가능하다

`-` 특정 노드와 연결된 색깔 종류 출력은 root의 색깔 집합의 길이이며 find 연산은 거의 $O(1)$이다

`-` 따라서 전체 알고리즘의 시간 복잡도는 $O(N \log N + Q)$이다

`-` 작은 집합에서 큰 집합으로 옮기므로 굳이 랭크 기반으로 유니온하지 않아도 된다

`-` 참고로 len 함수의 시간 복잡도는 $O(1)$이다

`-` 원소가 추가되거나 제거될 때마다 내부적으로 길이를 조정한다

`-` 즉, 호출할 때마다 전체 원소를 순회해서 길이를 구하는게 아니다 

In [5]:
import sys

sys.setrecursionlimit(10**5 + 2)


def find(u):
    if parents[u] != u:
        parents[u] = find(parents[u])
    return parents[u]


def union(u, v, color_sets):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if len(color_sets[u_root]) < len(color_sets[v_root]):
        small_to_large(color_sets[u_root], color_sets[v_root])
        parents[u_root] = v_root
    else:
        small_to_large(color_sets[v_root], color_sets[u_root])
        parents[v_root] = u_root


def small_to_large(small, large):
    large.update(small)
    small.clear()


def offline_query(queries, parent_nodes, color_sets):
    queries.reverse()
    answers = []
    for query in queries:
        operator, a = query
        if operator == 1:
            union(a, parent_nodes[a], color_sets)
        else:
            root = find(a)
            answer = len(color_sets[root])
            answers.append(answer)
    return answers


def solution():
    global parents
    N, Q = map(int, input().split())
    parent_nodes = [None for _ in range(N + 1)]
    for i in range(1, N):
        p = int(input())
        parent_nodes[i + 1] = p
    colors = [None for _ in range(N + 1)]
    for i in range(1, N + 1):
        c = int(input())
        colors[i] = c
    parents = [u for u in range(N + 1)]
    color_sets = [None] + [set([colors[i]]) for i in range(1, N + 1)]
    queries = [list(map(int, input().split())) for _ in range(N + Q - 1)]
    answers = offline_query(queries, parent_nodes, color_sets)
    print("\n".join(map(str, answers[::-1])))


solution()

# input
# 5 4
# 5
# 2
# 2
# 1
# 1
# 3
# 2
# 3
# 3
# 1 4
# 2 1
# 2 3
# 1 2
# 2 5
# 1 5
# 2 3
# 1 3

 5 4
 5
 2
 2
 1
 1
 3
 2
 3
 3
 1 4
 2 1
 2 3
 1 2
 2 5
 1 5
 2 3
 1 3


3
3
2
2


## 교수님은 기다리지 않는다

- 문제 출처: [백준 3830번](https://www.acmicpc.net/problem/3830)

`-` 유니온 파인드를 이용해 두 원소의 차이를 관리하는 문제이다

`-` 두 원소를 비교했다면 무게를 비교할 수 있으니 같은 그룹이다

`-` 같은 그룹에 속해있으면 무게를 비교할 수 있고 그룹 관리는 union-find로 하면 된다

`-` 무게 차이를 어떻게 측정할 것인지가 관건이다

`-` 각 그룹엔 루트 노드가 있고 find 함수를 통해 그룹의 루트를 찾는 건 쉽다

`-` 그룹의 루트와 자기 자신과의 무게 차이를 gaps 배열에 관리하자

`-` $\operatorname{gaps}[x] = w_{find(x)} - w_x$이다 (find는 그룹의 루트를 찾아준다)

`-` 즉, 그룹의 루트 무게에서 자기 무게를 차감한 것이다

`-` 이를 알면 같은 그룹에 속한 두 원소의 무게 차이를 계산할 수 있다

`-` 예컨대 $b - a = \operatorname{gaps}[a] - \operatorname{gaps}[b]$이다

`-` ? 쿼리는 gaps 배열만 정확하다면 쉽게 수행할 수 있다

`-` 초기엔 각 노드의 부모 노드를 자기 자신으로 설정하므로 $\operatorname{gaps}[x]$는 $0$이다

`-` ! 쿼리가 입력되었다고 하자

`-` 이는 어떤 원소 $u, v$가 있고 이들이 각각의 그룹에 속하는 것이다

`-` 두 그룹을 union해야 한다

`-` $w_v - w_u = w$라 하자

`-` $u$의 루트를 $u_r$, $v$의 루트를 $v_r$이라 하자

`-` 그럼 자명하게 $\operatorname{gaps}[u_r] = 0, \operatorname{gaps}[v_r] = 0$이다

`-` union by rank를 할 것이다, $\operatorname{ranks}[u_r] < \operatorname{ranks}[v_r]$이라 해보자

`-` 그럼 $\operatorname{parents}[u_r] = v_r$이 된다

`-` 근데 이러면 $u_r$ 그룹에 속한 원소들의 경우 루트가 바뀌고 gaps도 바뀐다

`-` $v_r$ 그룹에 속한 원소들은 바뀐게 없고 $u_r$에 속한 원소들만 업데이트하면 된다

`-` $u_r$ 그룹에 속한 원소들은 하루 아침에 루트가 $u_r$에서 $v_r$로 바뀌었다

`-` 이걸 반영해줘야 한다

`-` $w_v - w_u = w$일 때 $w_{vr} - w_{ur}$은 얼마일까?

`-` $w_{vr} - w_u = \operatorname{gaps}[v] + w, w_{ur} - w_u = \operatorname{gaps}[u]$

`-` $w_{vr} - w_{ur} = \operatorname{gaps}[v] - \operatorname{gaps}[u] + w = \operatorname{gaps}[w_{ur}]$

`-` 이렇게 하면 $\operatorname{gaps}[w_{ur}]$을 업데이트 해줄 수 있다

`-` 그런데 문제가 $u_r$ 그룹에 속한 모든 원소 $x$에 대해 $\operatorname{gaps}[x]$를 바꿔줘야 한다 (정의에 따라 루트가 바뀌었으니까)

`-` 즉, $\operatorname{gaps}[w_{ur}]$을 $\operatorname{gaps}[x]$에 더해줘서 루트가 $v_r$로 바뀐걸 반영해야 된다

`-` 하지만 이는 $u_r$ 그룹에 속한 원소의 수를 $N$이라 할 때 최악의 경우 $O(N)$의 시간 복잡도를 가진다

`-` 쿼리가 $M$개이므로 전체 알고리즘의 시간 복잡도는 $O(NM)$이니 시간 초과이다

`-` 따라서 ? 쿼리가 입력되어 $u$와 $v$에 find를 수행할 때 $u$와 $v$와 관련된 gaps를 업데이트 해줘야 된다

`-` 그룹의 루트는 바뀌었지만 $u_r$를 제외한 나머지 그룹 원소의 parents 배열은 수정되지 않았으며 gaps 배열도 마찬가지이다

`-` find 함수에서 루트를 찾아 부모를 거슬러 올라갈 때 $x$의 부모 $p$의 $\operatorname{gaps}[p]$에 대해 $\operatorname{gaps}[x]$에 이를 더해주면 된다

`-` 그럼 find 함수에서 $x$의 $\operatorname{parents}[x]$를 경로 압축을 통해 루트 노드로 바꾸면서 누적된 차이도 반영시키게 된다

`-` 무게 차이 관리를 union-find 알고리즘으로 해결하는게 신박한 문제였다

In [3]:
def find(u):
    if parents[u] != u:
        p = parents[u]
        parents[u] = find(parents[u])
        gaps[u] += gaps[p]
    return parents[u]


def union(u, v, w):
    # w_v - w_u = w
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if ranks[u_root] < ranks[v_root]:
        parents[u_root] = v_root
        gaps[u_root] = gaps[v] - gaps[u] + w
    elif ranks[v_root] < ranks[u_root]:
        parents[v_root] = u_root
        gaps[v_root] = gaps[u] - gaps[v] - w
    else:
        parents[u_root] = v_root
        gaps[u_root] = gaps[v] - gaps[u] + w
        ranks[v_root] += 1


def solve_testcase(n, m):
    global parents, ranks, gaps
    parents = [u for u in range(n + 1)]
    ranks = [0 for _ in range(n + 1)]
    gaps = [0 for _ in range(n + 1)]
    for _ in range(m):
        query = list(input().split())
        operator = query[0]
        if operator == "!":
            _, a, b, w = query
            a, b, w = int(a), int(b), int(w)
            union(a, b, w)
        else:
            _, a, b = query
            a, b = int(a), int(b)
            if find(a) != find(b):
                print("UNKNOWN")
            else:
                print(gaps[a] - gaps[b])


def solution():
    while True:
        n, m = map(int, input().split())
        if n == m == 0:
            break
        solve_testcase(n, m)


solution()

# input
# 2 2
# ! 1 2 1
# ? 1 2
# 2 2
# ! 1 2 1
# ? 2 1
# 4 7
# ! 1 2 100
# ? 2 3
# ! 2 3 100
# ? 2 3
# ? 1 3
# ! 4 3 150
# ? 4 1
# 0 0

 2 2
 ! 1 2 1
 ? 1 2


1


 2 2
 ! 1 2 1
 ? 2 1


-1


 4 7
 ! 1 2 100
 ? 2 3


...

 ? 1 3


200


 ! 4 3 150
 ? 4 1


-50


 0 0


## 산책과 쿼리

- 문제 출처: [백준 28121번](https://www.acmicpc.net/problem/28121)

`-` 유니온 파인드를 이용해 이분 그래프를 관리하는 문제 $2$, 추가로 태그도 봤다 (애드 혹...)

`-` 산책로 리스트 중 마음대로 골라서 이동 가능하다

`-` 원래 장소로 임의의 시각 $t(\le 10^6)$에 돌아오면 된다

`-` 임의의 장소 $x$에서 다시 돌아오는 시간이 홀수이냐 짝수이냐에 따라 갈린다

`-` 둘 다 가능해야 만족스러운 산책이다 (짝수, 홀수 둘 다 가능하면 $10^6$을 넘는 모든 $t$에 대해 가능함)

`-` $N$에 비해 $t$가 충분히 크니 임의의 짝수 시간, 홀수 시간이 걸리는 경로가 $10^6$시간을 넘지 못할까 걱정하지 않아도 된다

`-` $N \le 3\cdot 10^5$이므로 $3$바퀴 돌아도 $10^6$시간이 안된다

`-` 일단 만족스러운 산책이 가능하려면 경로에 사이클이 있어야 한다

`-` 사이클이 없다면 경로는 $x \to a \to x$이고 산책로는 양방향이므로 $x \to a$와 $a \to x$는 동일한 시간이 걸린다

`-` 이는 짝수 시간이므로 만족스러운 산책이 아니다 (홀수 시간은 불가능)

`-` 짝수, 홀수 판단을 위해 이분 그래프로 관리할 것이다 (장소 노드에 $0$ 또는 $1$로 마킹을 하자)

`-` 산책로를 만드는 건 두 장소를 union한다는 것이다

`-` union할 때 두 원소 모두 마크가 없다면 임의의 원소를 $0$, 나머지를 $1$로 마킹한다

`-` 한 곳에만 마크가 없다면 없는 쪽에 있는 쪽과 반대로 마킹을 하면 된다

`-` 참고로 현재 장소와 연결된 장소로 움직이면 시간이 $1$ 증가하고 이는 짝수와 홀수를 바꾸므로 반대 마킹을 해야 된다

`-` 두 원소 모두 마킹됐다고 하자

`-` 그런데 두 원소의 마크가 동일하면 모순이다

`-` 이는 산책 시간으로 짝수, 홀수 시간 둘 다 가능하다는 것을 의미한다

`-` 산책로 내 한 곳이라도 만족스러운 산책이 가능한 장소가 있다면 산책로 내 모든 장소에서 만족스러운 산책이 가능하다

`-` $q$가 산책의 자유도가 높다면 $x \to q \to x$에서 $q -> x$는 짝수 시간과 홀수 시간 둘 다 가능하므로 $x$ 역시 산책의 자유도가 높은 것이다

`-` 한편, 두 장소를 union할 때 서로 다른 산책로에 속하면 주의할 사항이 있다

`-` 두 원소의 마크가 다르면 괜찮은데 같다면 그룹 사이즈가 작은 쪽 원소의 마크를 모두 반대로 설정한 후 union해야 한다

`-` 다른 그룹을 하나로 합친 것이니 만족스러운 산책이 가능한 건 아니다

`-` 합친 후에 산책로 내에 사이클이 존재하지 않는다 (존재한다면 애초에 다른 그룹일 수 없다)

`-` 마킹은 산책로 내의 상대적인 위치를 추상화한 것이고 이를 $0$과 $1$로 나타낸 것이다 (다른 산책로와 연관되지 않음)

`-` small to large로 수행하면 시간 복잡도는 $O(N \log N)$이다

`-` 위를 실현시키기 위해 union-find에서 루트 노드는 자기 집합의 모든 원소를 가지고 있어야 한다

`-` union할 때 만족스러운 산책이 가능하면 루트 노드에 이를 저장하자

`-` union할 때 어느 한쪽의 루트 노드라도 만족스러운 산책이 가능하면 두 그룹 모두 무조건 가능이다

`-` 만족스러운 산책이 가능한 장소의 수는 만족스러운 산책이 가능한 그룹에 속한 원소의 총 개수이다

`-` 산책로 추가될 때마다 새로 생긴 만족스러운 산책 장소 개수 정답에 누적하면 된다

In [1]:
def find(u):
    if parents[u] != u:
        parents[u] = find(parents[u])
    return parents[u]


def union(u, v):
    global ANSWER
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        if good_walks[u_root]:
            return
        if marks[u] == marks[v]:
            good_walks[u_root] = True
            ANSWER += len(element_sets[u_root])
        return
    size_u = len(element_sets[u_root])
    size_v = len(element_sets[v_root])
    if good_walks[u_root] and good_walks[v_root]:
        if size_u < size_v:
            parents[u_root] = v_root
            small_to_large(element_sets[u_root], element_sets[v_root])
        else:
            parents[v_root] = u_root
            small_to_large(element_sets[v_root], element_sets[u_root])
        return
    if good_walks[u_root]:
        if size_u < size_v:
            parents[u_root] = v_root
            small_to_large(element_sets[u_root], element_sets[v_root])
        else:
            parents[v_root] = u_root
            small_to_large(element_sets[v_root], element_sets[u_root])
        ANSWER += size_v
        return
    if good_walks[v_root]:
        if size_u < size_v:
            parents[u_root] = v_root
            small_to_large(element_sets[u_root], element_sets[v_root])
        else:
            parents[v_root] = u_root
            good_walks[u_root] = True
            small_to_large(element_sets[v_root], element_sets[u_root])
        ANSWER += size_u
        return
    if marks[u] != marks[v]:
        if size_u < size_v:
            parents[u_root] = v_root
            small_to_large(element_sets[u_root], element_sets[v_root])
        else:
            parents[v_root] = u_root
            small_to_large(element_sets[v_root], element_sets[u_root])
    else:
        if size_u < size_v:
            parents[u_root] = v_root
            reverse_mark(element_sets[u_root])
            small_to_large(element_sets[u_root], element_sets[v_root])
        else:
            parents[v_root] = u_root
            reverse_mark(element_sets[v_root])
            small_to_large(element_sets[v_root], element_sets[u_root])


def small_to_large(small, large):
    large.update(small)
    small.clear()


def reverse_mark(elements):
    for e in elements:
        marks[e] = 1 - marks[e]


def solution():
    global ANSWER, parents, marks, element_sets, good_walks
    ANSWER = 0
    N, Q = map(int, input().split())
    parents = [u for u in range(N + 1)]
    marks = [0 for _ in range(N + 1)]
    element_sets = [set([u]) for u in range(N + 1)]
    good_walks = [False for _ in range(N + 1)]
    for _ in range(Q):
        a, b = map(int, input().split())
        union(a, b)
        print(ANSWER)


solution()

# input
# 6 5
# 1 2
# 2 3
# 1 3
# 4 5
# 2 5

 6 5
 1 2


0


 2 3


0


 1 3


3


 4 5


3


 2 5


5
